<a href="https://colab.research.google.com/github/sethgis/dash_front_end/blob/main/Copy_of_EditDNBR_alerts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 **Install Packages and Import them**

In [ ]:
# Installs geemap package
import subprocess
 
try:
    import geemap
    import geopandas as gpd
    import numpy as np
    import folium
    import rasterio
    import tensorflow
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geopandas'])
    subprocess.check_call(["python", '-m', 'pip', 'install', 'folium'])
    subprocess.check_call(["python", '-m', 'pip', 'install', "rasterio"])
    subprocess.check_call(["python", '-m', 'pip', 'install', "tensorflow"])

    
 
# Checks whether this notebook is running on Google Colab
try:
    import google.colab
    import geemap.eefolium as geemap
except:
    import geemap
 
# Authenticates and initializes Earth Engine
import ee
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm, gamma, f, chi2
import IPython.display as disp
import geopandas as gpd
from shapely.geometry import Point, Polygon
from geemap import geojson_to_ee, ee_to_geojson
from geopandas import GeoSeries
import json
import pandas as pd
%matplotlib inline 

In [ ]:
def authenticate():
  try:
    ee.Initialize()
  except Exception as e:
    ee.Authenticate()
    ee.Initialize()

In [ ]:
authenticate()

In [ ]:
AOI = ee.FeatureCollection('users/snyawacha/Il_NgwesiConservancy')
AOI = AOI
START_DATE = '2019-04-01'
END_DATE = '2019-09-01'
START_DATE2 = '2020-04-01'
END_DATE2 = '2020-09-01'
CLOUD_FILTER = 60
CLD_PRB_THRESH = 40
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 2
BUFFER = 100


In [ ]:
#Area of interest
ROI = ee.FeatureCollection("users/collinsasegaca/TROFMIS_Forest_specific_AOI");
LOOKS = 5 
BASE_PERIOD = ['2020-01-01' ,'2020-03-15']
BASE_PERIOD = ['2020-01-01' ,'2020-03-15']
BASE_PERIOD = ['2020-01-01' ,'2020-03-15']
 # The before Image Date
ANALYSIS_PERIOD = ['2020-03-16' ,'2020-06-29'] # The after Image Date
COUNTRIES = ["KENYA"]#, "UGANDA"] #countries of interest
ALL_COUNTRIES = "IGAD"

In [ ]:
def get_s2_sr_cld_col(aoi, start_date):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(aoi)
        .filterDate(start_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

s2_sr_cld_col_eval = get_s2_sr_cld_col(AOI, BASE_PERIOD)
s2_sr_cld_col_eval2 = get_s2_sr_cld_col(AOI, ANALYSIS_PERIOD )


def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))

def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focal_min(2).focal_max(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)


def display_cloud_layers(col):
    # Mosaic the image collection.
    img = col.mosaic()

    # Subset layers and prepare them for display.
    clouds = img.select('clouds').selfMask()
    shadows = img.select('shadows').selfMask()
    dark_pixels = img.select('dark_pixels').selfMask()
    probability = img.select('probability')
    cloudmask = img.select('cloudmask').selfMask()
    cloud_transform = img.select('cloud_transform')

    image = s2_sr_cld_col_eval.map(add_cld_shdw_mask)
    image2 = s2_sr_cld_col_eval2.map(add_cld_shdw_mask)

In [ ]:
kernel_size = 4;
min_disturbances = 0.05;
kernel_clean_size = 10;
threshold_conservative = 0.04;

**Image Analysis **

In [ ]:
def _get_radar_alert(aoi):
  """# Function takes in Region of interest base period and comparison period and returns two products loss and gain products
  """
  #  Obtain the Image Collection  In the Descending orbit Pass
  collection = (ee.ImageCollection('COPERNICUS/S2')
                 .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 20)) filters on the metadata for pixels less than 10% cloud
                 .map(maskS2clouds)//... chooses only pixels between the dates you define here
                 .filterBounds(table);
                 .filterBounds(ROI))
  
  #filter before and after collection according to the defined Dates 
  before_collection = collection.filterDate(BASE_PERIOD[0], BASE_PERIOD[1])
  after_collection = collection.filterDate(ANALYSIS_PERIOD[0], ANALYSIS_PERIOD[1])

  #getting before and after image as well as Clipping

  im1 = before_collection.mosaic().clip(aoi) #.resample('bicubic')
  im2 = after_collection.mosaic().clip(aoi) #.resample('bicubic') 


  # Apply speckle a filter
  def smoothen(image1, image2):
    smoothing_radius = 30;
    im1 = image1.focal_median(smoothing_radius, 'circle', 'meters')
    im2 = image2.focal_median(smoothing_radius, 'circle', 'meters')
    return im1, im2

  im1_smooth = smoothen(im1,im2)
  im1 = im1_smooth[0]
  im2 = im1_smooth[1]

  #  Likelihood and Statistical Testing.

  def det(im):
    return im.expression('b(0) * b(1)')

  # Number of looks.
  m = LOOKS
  # aoi = roi

  m2logQ = det(im1).log().add(det(im2).log()).subtract(
      det(im1.add(im2)).log().multiply(2)).add(4*np.log(2)).multiply(-2*m)


  def chi2cdf(chi2, df):
    ''' Chi square cumulative distribution function for df degrees of freedom
        using the built-in incomplete gamma function gammainc() '''
    return ee.Image(chi2.divide(2)).gammainc(ee.Number(df).divide(2))

  # The observed test statistic image -2logq.
  m2logq = det(im1).log().add(det(im2).log()).subtract(
      det(im1.add(im2)).log().multiply(2)).add(4*np.log(2)).multiply(-2*m)

  # The P value image prob(m2logQ > m2logq) = 1 - prob(m2logQ < m2logq).
  p_value = ee.Image.constant(1).subtract(chi2cdf(m2logq, 2))

  c_map1 = p_value.multiply(0).where(p_value.lt(0.05), 1)
  diff = im2.subtract(im1)
  d_map = c_map1.multiply(0)                    # Initialize the direction map to zero.
  d_map = d_map.where(det(diff).gt(0), 2)      # All pos or neg def diffs are now labeled 2.
  d_map = d_map.where(diff.select(0).gt(0), 3) # Re-label pos def (and label some indef) to 3.
  d_map = d_map.where(det(diff).lt(0), 1)      # Label all indef to 1.
  c_map1 = c_map1.multiply(d_map)                # Re-label the c_map, 0*X = 0, 1*1 = 1, 1*2= 2, 1*3 = 3.

  c_map_loss = c_map1.updateMask(c_map1.eq(2)).rename('loss') #loss
  c_map_gain = c_map1.updateMask(c_map1.eq(3)).rename('gain') #gain

  return [c_map_loss, c_map_gain]


In [ ]:
def compute_regional_radar_alert(return_url=False):
  """
  Get radar alerts at regional level
  genre"""
  return compute_country_radar_alert(country_name=ALL_COUNTRIES, return_url=return_url)


In [ ]:
def compute_country_radar_alert(country_name, return_url=True):
  """
  Get radar alerts at country level
  Returns a  of (alert, loss_url, gain_url)
  """
  aoi = filter_region(country_name=country_name)
  alerts = _get_radar_alert(aoi=aoi)
  if return_url:
    return (alerts, 
            get_url(alert=alerts[0],
                      file_name=generate_file_name(country_name, "loss"), 
                      vector=aoi), #loss url
            get_url(alert=alerts[1],
                      file_name=generate_file_name(country_name, "gain"), 
                      vector=aoi) #gain url
           )
  return (alert, None, None)

Funtion for generation of alerts per country and their Urls, 
# `*Note: the scale is high therefore the image resolution is redused *`

In [ ]:
def alert_url_generator(country_name, activity):
  """
  If you need a url at regional level, pass country_name=ALL_COUNTRIES
  """
  country_alert = country_radar_alert(country)  
  country_url = get_url(alert=country_alert[0] if activity.title()=="Loss" else country_alert[1],
                         file_name=generate_file_name(country_name, activity),
                         vector=country.geometry())
  return country_url 

In [ ]:
def generate_file_name(country_name, activity):
  return "%s_to_%s_%s_%s.tif" % (BASE_PERIOD[0], ANALYSIS_PERIOD[1],
                                   country_name, activity)

In [ ]:
def get_url(alert, file_name, vector):
  """
  Will export a tif and return the url
  """
  # exported_img = geemap.ee_export_image(
  #                     ee_object=alert,
  #                     # country_alert[0] if activity.title()=="Loss" else country_alert[1], 
  #                     filename=file_name, 
  #                     scale=200,
  #                     region = vector.geometry(),
  #                     file_per_band=False)
  url = alert.getDownloadUrl({
				'scale': 200, 			
				'crs': 'EPSG:4326', 
				'region': vector.geometry()
			}) 
  return url

In [ ]:
# All_url = alert_Url_generator(ROI, 'IGAD', 'Loss', BASE_PERIOD, ANALYSIS_PERIOD)
# regional_url = alert_url_generator(country_name=ALL_COUNTRIES) # Url for entire region

**Plot the Map of the Products **

In [ ]:
# call the function for the entire trofmis AoI
# Trofmis_alert = regional_radar_alert(ROI, BASE_PERIOD,ANALYSIS_PERIOD)

In [ ]:
def filter_region(country_name):
  """Filter regional vector by country"""
  if country_name != ALL_COUNTRIES:
    return ROI.filter(ee.Filter.eq('COUNTRY', country_name.upper()))
  return ROI

In [ ]:
# Get alerts for the countries of interest
country_alerts = []
for i, cntry in enumerate(COUNTRIES): 
  country_alerts.append(compute_country_radar_alert(cntry))# Appends a tuple of ((loss, gain), loss_url, gain_url)

print ("Country urls: ", [list(x)[1:] for x in country_alerts])


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/lib/python3.7/bdb.py", line 332, in set_trace
    sys.settrace(self.trace_dispatch)



> <ipython-input-22-f4bb50d261b1>(18)get_url()
-> return url
(Pdb) alert.bandNames
<bound method ApiFunction.importApi.<locals>.MakeBoundFunction.<locals>.<lambda> of <ee.image.Image object at 0x7f3e65dd7250>>
(Pdb) vector
(Pdb) url = alert.getDownloadUrl({ 				'scale': 200, 			 				'crs': 'EPSG:4326', })
*** ee.ee_exception.EEException: Pixel grid dimensions (200376x100188) must be less than or equal to 10000.
(Pdb) q



PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/lib/python3.7/bdb.py", line 357, in set_quit
    sys.settrace(None)



BdbQuit: ignored

In [ ]:
country_alerts

[([<ee.image.Image at 0x7f3e66235310>, <ee.image.Image at 0x7f3e6621cd50>],
  'https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/476ecac66bf3d62bda3691c136624589-9cfbf32d2a7a5c88fe18f30e2bfa1aa4:getPixels',
  'https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/1ccc15a2e42e5e7b3f3340e3aab48f7a-031f1bb19c0d99f31948d2b6a9323fbb:getPixels')]

In [ ]:
Map = geemap.Map()

Map.add_basemap('SATELLITE')
Map.centerObject(ROI, 12)

# for alert in country_alerts:
Map.addLayer(s2_sr_cld_col_eval_disp,{'min': 0,'max': 1, 'palette': ['red']}, 'loss')
Map.addLayer(s2_sr_cld_col_eval_disp2,{'min': 0,'max': 1, 'palette': ['blue']}, 'gain')
Map

# Map = geemap.Map()

# Map.add_basemap('SATELLITE')
# Map.centerObject(roi,12)

# Map.addLayer(Trofmis_alert[0],{'min': 0,'max': 1, 'palette': ['red']}, 'loss')
# Map.addLayer(Trofmis_alert[1],{'min': 0,'max': 1, 'palette': ['blue']}, 'gain')
# Map

NameError: ignored

In [ ]:
# task = ee.batch.Export.image.toDrive(image=c_map_gain,  # an ee.Image object.
#                                      region=roi.geometry(),  # an ee.Geometry object.
#                                      description='Kakamega_radar',
#                                      folder='Radar_changes',
#                                      fileNamePrefix='Kakamega_Gain_Feb_March_Image',
#                                      scale=10,
#                                      crs='EPSG:4326')

In [ ]:
# task.start()

In [ ]:
# task.status()

In [ ]:
# geemap.ee_export_image(c_map_loss, filename='Kakamega_loss_radar.tif',scale=10, region= AOI.geometry(),file_per_band=False)